# Классификация изображений

ноутбук для решения задания [Всероссийской олимпиады по искусственному интеллекту](https://ai.edu.gov.ru/) "Классификация изображений".

### 1. Исследование данных:

In [1]:
import pandas as pd

Train = pd.read_csv('Train.csv', sep=';')

# Отбор нужных данных:
class_names = ['Торговля и объявления','Животные','Кулинария','Творчество и дизайн',
               'Развлечения и юмор', 'СМИ', 'Философия и религия','Путешествия']
Train = Train[Train.label.isin(class_names)].drop(columns=['description']).sample(frac=1, random_state=42)

label2id = {label: i for i, label in enumerate(class_names)}
id2label = {i: label for label, i in label2id.items()}
Train['label'] = Train.label.map(label2id)

# Выделение валидационной выборки
val = pd.concat([Train[Train.label == c].iloc[:20] for c in range(8)])
Train = Train.loc[~Train.index.isin(val.index)]
val.shape, Train.shape

((160, 2), (4787, 2))

In [2]:
# Проверка баланса классов:
WC = Train.groupby(Train.label).count()
WC[['id']].T

label,0,1,2,3,4,5,6,7
id,669,420,775,311,1376,498,404,334


### 2. Для устранения дисбаланса классов

In [3]:
WC['wc'] = (WC.id.max() / WC.id)
WC[['wc']].T

# # Веса для семплирования мини-партий данных
WfS = Train.label.map(WC.wc.to_dict())

In [4]:
# Для расширения данных
import albumentations as A
from albumentations.pytorch import ToTensorV2

transform = A.Compose([
    A.Resize(height=480, width=480, p=1),   
    A.HorizontalFlip(p=.33),
    A.RandomBrightnessContrast(brightness_limit=(-0.2, 0.4), contrast_limit=(-0.3, 0.4), p=0.5),
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.25, rotate_limit=30, p=0.4),
    A.Blur(blur_limit=3),
    A.OpticalDistortion(),
    A.GridDistortion(),
    A.HueSaturationValue(),
    A.Normalize(),
    ToTensorV2(),
])

INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.12 (you have 1.4.11). Upgrade using: pip install --upgrade albumentations


### 3. Выбор модели и адаптация к задаче

In [5]:
from torchvision.models import efficientnet_v2_m, EfficientNet_V2_M_Weights
import torch
from torch import nn

torch.manual_seed(42)

weights = EfficientNet_V2_M_Weights.IMAGENET1K_V1
model = efficientnet_v2_m(weights=weights)
preprocess = weights.transforms()

# замарозка весов блоков, короме несколько последних слоев, в части извлечения признаков
for params in model.features[:-2].parameters():
    params.requires_grad = False
for params in model.features[-2:].parameters():
    params.requires_grad = True

# изменения в слоях 
model.features[-1][0] = nn.Conv2d(512, 256, (1,1))
model.features[-1][1] = nn.BatchNorm2d(256)
model.classifier = nn.Sequential(
                                nn.Dropout(0.3),
                                nn.Linear(256, 128),
                                nn.SELU(),
                                nn.Dropout(0.3),
                                nn.Linear(128, 8))

model.to('cuda');

### 4. Обучение модели

In [6]:
from torchvision.transforms import ToTensor
import cv2
from torcheval.metrics.functional import multiclass_f1_score as F1

# Гиперпараметры обучения модели 
lr, d, m, bs, cp = 3e-4, .66, 1.02, 64, 36
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
loss_fn = torch.nn.CrossEntropyLoss(label_smoothing=.01)

# Для валидации
valX = torch.stack([preprocess(ToTensor()(cv2.cvtColor(cv2.imread(f'Train/{x}'), cv2.COLOR_BGR2RGB)))
                    for x in val.id]).to('cuda')
valY = torch.tensor(val.label.values).to('cuda')

# Для контроля переобучения
l1, l0 = 10, 10

In [7]:
for step in range(1, 1000):
        model.train()
        batch = Train.sample(n=bs, random_state=step, weights=WfS)
        X = torch.stack(batch.id.map(lambda x: 
                    transform(image=cv2.cvtColor(cv2.imread(f'Train/{x}'),
                                    cv2.COLOR_BGR2RGB))['image']).to_list()).to('cuda')
        y = torch.tensor(batch.label.values).to('cuda')
        optimizer.zero_grad()
        pred = model(X)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()
        print(f'\rTrain: {-step%cp:2} Loss: {loss:.4f} F1: {F1(pred.argmax(1), y, num_classes=8).item():.4f}\t', end='')
        if step % cp: continue
        model.eval()
        with torch.no_grad():
                pred = model(valX)
                loss = loss_fn(pred, valY).mean().item()
                print(f"Val {step//cp}:\tLoss: {loss:.4f}",
                        f'F1: {F1(pred.argmax(1), valY, num_classes=8).item():.4f}',
                        f'lr: {optimizer.param_groups[0]["lr"]:.7f}', sep='\t')
                if loss > l0 and l1 > l0: break
                l1, l0 = loss, l1
                lr *= d; d *= m
                optimizer.param_groups[0]["lr"] = lr

Train:  0 Loss: 1.0819 F1: 0.7656	Val 1:	Loss: 0.9530	F1: 0.7125	lr: 0.0003000
Train:  0 Loss: 0.9144 F1: 0.7656	Val 2:	Loss: 0.8385	F1: 0.7688	lr: 0.0001980
Train:  0 Loss: 0.8762 F1: 0.6719	Val 3:	Loss: 0.7366	F1: 0.7750	lr: 0.0001333
Train:  0 Loss: 0.6036 F1: 0.8594	Val 4:	Loss: 0.7280	F1: 0.7750	lr: 0.0000915
Train:  0 Loss: 0.6468 F1: 0.8281	Val 5:	Loss: 0.6989	F1: 0.7937	lr: 0.0000641
Train:  0 Loss: 0.4479 F1: 0.8750	Val 6:	Loss: 0.7035	F1: 0.7812	lr: 0.0000458
Train:  0 Loss: 0.5260 F1: 0.8125	Val 7:	Loss: 0.6943	F1: 0.8000	lr: 0.0000334
Train:  0 Loss: 0.3851 F1: 0.8750	Val 8:	Loss: 0.6990	F1: 0.8062	lr: 0.0000248
Train:  0 Loss: 0.4335 F1: 0.8594	Val 9:	Loss: 0.7078	F1: 0.7812	lr: 0.0000188


### 5. Делаем предсказание на тестовой выборке

In [8]:
from tqdm import tqdm

test = pd.read_csv('Test.csv', sep=';')
model.eval()
y = []
with torch.no_grad():
        for i in tqdm(range(0, len(test), bs)):
                X = torch.stack(test.iloc[i: i+bs].id.map(
                        lambda x: preprocess(ToTensor()(
                                cv2.cvtColor(cv2.imread(f'Test/{x}'), cv2.COLOR_BGR2RGB)
                                ))).to_list()).to('cuda')
                y += model(X).argmax(1).tolist()

100%|██████████| 41/41 [07:54<00:00, 11.57s/it]


In [9]:
test['label'] = y
test.replace(id2label, inplace=True)
sub = test.drop(columns=['description']).set_index('id')
sub.to_csv('vk_submission_ef-4.csv', index='True', sep = ';')
sub

,label
id,
909340245742,Развлечения и юмор
909342962411,Развлечения и юмор
909343087161,Путешествия
909344193109,Торговля и объявления
909346841420,Кулинария
...,...
970656513024,Путешествия
970691145216,Творчество и дизайн
970699981568,Путешествия
